<a href="https://colab.research.google.com/github/Het0c/Ep4Machine/blob/main/Descripcion_y_limpieza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # Para gráficos 3D
from sklearn.linear_model import LinearRegression
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import os
from google.colab import files

# Cargar datos

if not os.path.exists("Anexo_ET_demo_round_traces_2022.csv"):
  uploaded = files.upload()
# 1. Lectura
df = pd.read_csv('Anexo_ET_demo_round_traces_2022.csv', sep=";")

# 2. Dimensiones
print("Dimensiones (filas, columnas):", df.shape)

# 3. Tipos de datos y faltantes
print("\nTipos de datos:\n", df.dtypes)
print("\nValores nulos por columna:\n", df.isnull().sum())

# 4. Estadísticas descriptivas
display(df.describe(include='all'))


Saving Anexo_ET_demo_round_traces_2022.csv to Anexo_ET_demo_round_traces_2022.csv


/tmp/ipython-input-2-4266535830.py:18: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Anexo_ET_demo_round_traces_2022.csv', sep=";")


Dimensiones (filas, columnas): (79157, 30)

Tipos de datos:
 Unnamed: 0                       int64
Map                             object
Team                            object
InternalTeamId                   int64
MatchId                          int64
RoundId                          int64
RoundWinner                     object
MatchWinner                     object
Survived                          bool
AbnormalMatch                     bool
TimeAlive                       object
TravelledDistance               object
RLethalGrenadesThrown            int64
RNonLethalGrenadesThrown         int64
PrimaryAssaultRifle            float64
PrimarySniperRifle             float64
PrimaryHeavy                   float64
PrimarySMG                     float64
PrimaryPistol                    int64
FirstKillTime                   object
RoundKills                       int64
RoundAssists                     int64
RoundHeadshots                   int64
RoundFlankKills                  int64
Rou

,Unnamed: 0,Map,Team,InternalTeamId,MatchId,RoundId,RoundWinner,MatchWinner,Survived,AbnormalMatch,...,RoundKills,RoundAssists,RoundHeadshots,RoundFlankKills,RoundStartingEquipmentValue,TeamStartingEquipmentValue,MatchKills,MatchFlankKills,MatchAssists,MatchHeadshots
count,79157.000000,79157,79155,79157.000000,79157.000000,79157.000000,79157,79156,79157,79157,...,79157.000000,79157.000000,79157.000000,79157.000000,79157.000000,79157.000000,79157.000000,79157.000000,79157.000000,79157.000000
unique,NaN,4,2,NaN,NaN,NaN,5,2,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,de_inferno,Terrorist,NaN,NaN,NaN,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,28869,39591,NaN,NaN,NaN,23195,42017,47214,79157,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,60570.193640,NaN,NaN,1.500006,250.525033,12.765517,NaN,NaN,NaN,NaN,...,0.671349,0.146178,0.301553,0.095633,3778.108695,18890.133532,8.511578,1.209912,1.874831,3.858484
std,36436.598386,NaN,NaN,0.500003,149.557804,7.432837,NaN,NaN,NaN,NaN,...,0.881798,0.385601,0.578843,0.320641,2111.285406,9743.203024,6.195079,1.435150,1.824027,3.246752
min,600.000000,NaN,NaN,1.000000,4.000000,1.000000,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,30830.000000,NaN,NaN,1.000000,128.000000,6.000000,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,1600.000000,10900.000000,3.000000,0.000000,0.000000,1.000000
50%,56720.000000,NaN,NaN,2.000000,236.000000,12.000000,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,4700.000000,22550.000000,8.000000,1.000000,1.000000,3.000000
75%,90639.000000,NaN,NaN,2.000000,373.000000,19.000000,NaN,NaN,NaN,NaN,...,1.000000,0.000000,0.000000,0.000000,5200.000000,26050.000000,13.000000,2.000000,3.000000,6.000000


Deteccion de datos invalidos "TimeAlive"

In [3]:
ta = df['TimeAlive'].astype(str)
cleaned = ta.str.replace('.', '', regex=False)
mask_null      = ta.isnull()
mask_empty     = ta.str.strip()==''
mask_not_num   = ~cleaned.str.isdigit()
mask_zero      = pd.to_numeric(cleaned, errors='coerce')==0
mask_invalid   = mask_null | mask_empty | mask_not_num | mask_zero

print(f"NULL/NaN: {mask_null.sum()}")
print(f"Cadenas vacías: {mask_empty.sum()}")
print(f"No numérico: {mask_not_num.sum()}")
print(f"== 0 luego de limpiar: {mask_zero.sum()}")
print(f"Total inválidos: {mask_invalid.sum()}")
print("\nEjemplos inválidos:\n", df.loc[mask_invalid, ['MatchId','RoundId','TimeAlive']].head(10))
print("\nValores únicos inválidos:", df.loc[mask_invalid,'TimeAlive'].unique())


NULL/NaN: 0
Cadenas vacías: 0
No numérico: 0
== 0 luego de limpiar: 68
Total inválidos: 68

Ejemplos inválidos:
      MatchId  RoundId TimeAlive
231        6        1       0.0
232        6        1       0.0
236        6        1       0.0
238        6        1       0.0
239        6        1       0.0
264        6        4       0.0
269        6        4       0.0
274        6        5       0.0
279        6        5       0.0
284        6        6       0.0

Valores únicos inválidos: ['0.0']


In [4]:
# Lista de columnas booleanas u object con True/False
bool_cols = ['RoundWinner', 'MatchWinner', 'Survived', 'AbnormalMatch']

# Método 1: astype(int) — funciona si pandas las reconoce ya como bool
# Remove the problematic line: df[bool_cols] = df[bool_cols].astype(int)

# Método 2: map() — para object con cadenas
mapping = {'True': 1, 'False': 0, 'true': 1, 'false': 0, True: 1, False: 0}
for col in bool_cols:
    # Use the mapping method to convert column values to integers
    df[col] = df[col].map(mapping)

# Verificación
print(df[bool_cols].dtypes)
display(df[bool_cols].head())


RoundWinner      float64
MatchWinner      float64
Survived           int64
AbnormalMatch      int64
dtype: object


,RoundWinner,MatchWinner,Survived,AbnormalMatch
0,0.0,1.0,0,0
1,0.0,1.0,0,0
2,0.0,1.0,0,0
3,0.0,1.0,0,0
4,0.0,1.0,1,0


In [12]:
df.head()

,Unnamed: 0,Map,Team,InternalTeamId,MatchId,RoundId,RoundWinner,MatchWinner,Survived,AbnormalMatch,...,RoundKills,RoundAssists,RoundHeadshots,RoundFlankKills,RoundStartingEquipmentValue,TeamStartingEquipmentValue,MatchKills,MatchFlankKills,MatchAssists,MatchHeadshots
0,600,de_inferno,Terrorist,1,4,1,0.0,1.0,0,0,...,0,0,0,0,750,4400,0,0,0,0
1,601,de_inferno,Terrorist,1,4,1,0.0,1.0,0,0,...,0,0,0,0,800,4400,0,0,0,0
2,602,de_inferno,Terrorist,1,4,1,0.0,1.0,0,0,...,0,0,0,0,1000,4400,0,0,0,0
3,603,de_inferno,Terrorist,1,4,1,0.0,1.0,0,0,...,0,0,0,0,850,4400,0,0,0,0
4,604,de_inferno,Terrorist,1,4,1,0.0,1.0,1,0,...,0,0,0,0,1000,4400,0,0,0,0


In [5]:
import pandas as pd
import numpy as np

# —————————————————————————————
# 1) PARSE y NORMALIZA TimeAlive a segundos
# —————————————————————————————
# Asumiendo formatos como “mm:ss” o “ss.s” o “m.s” en tu CSV:
def parse_time(val):
    s = str(val).strip()
    if pd.isna(val) or s == '' or not any(c.isdigit() for c in s):
        return np.nan
    # Si viene en “m:s” o “mm:ss”
    if ':' in s:
        mins, secs = s.split(':', 1)
        try:
            return int(mins)*60 + float(secs)
        except:
            return np.nan
    # Si viene en “123.45” (con punto decimal)
    try:
        return float(s)
    except:
        return np.nan

df['TimeAlive_seconds'] = df['TimeAlive'].apply(parse_time)

# —————————————————————————————
# 2) DETECCIÓN de outliers en TimeAlive
# —————————————————————————————
min_time, max_time = 10, 155
mask_invalid = (
    df['TimeAlive_seconds'].isna() |
    (df['TimeAlive_seconds'] < min_time) |
    (df['TimeAlive_seconds'] > max_time)
)

# Estadísticas de inválidos
n_invalid = mask_invalid.sum()
print(f"TimeAlive inválido antes de imputar: {n_invalid} filas")

# —————————————————————————————
# 3) CÁLCULO de promedios y mediana fallback
# —————————————————————————————
# Promedio válido por equipo en cada ronda
avg_team = (
    df.loc[~mask_invalid]
      .groupby(['MatchId','RoundId','InternalTeamId'])['TimeAlive_seconds']
      .mean()
      .rename('avg_time_by_team')
)

# Mediana global de validos
mediana_global = df.loc[~mask_invalid, 'TimeAlive_seconds'].median()

# —————————————————————————————
# 4) IMPUTACIÓN
# —————————————————————————————
fallback_count = 0
def imputa_time(row):
    global fallback_count
    t = row['TimeAlive_seconds']
    if pd.isna(t) or t < min_time or t > max_time:
        key = (row['MatchId'], row['RoundId'], row['InternalTeamId'])
        if key in avg_team.index:
            return avg_team.loc[key]
        else:
            fallback_count += 1
            return mediana_global
    return t

df['TimeAlive_clean'] = df.apply(imputa_time, axis=1)

# Verificación final
print(f"Usaron fallback global: {fallback_count} filas")
remaining = ((df['TimeAlive_clean'] < min_time) | (df['TimeAlive_clean'] > max_time) | df['TimeAlive_clean'].isna()).sum()
print(f"Outliers restantes tras imputación: {remaining}\n")

# —————————————————————————————
# 5) LIMPIEZA de TravelledDistance
# —————————————————————————————
# 5.1 Convierte a numérico (elimina comas, signos)
df['TravelledDistance_num'] = (
    df['TravelledDistance']
      .astype(str)
      .str.replace(',', '', regex=False)
      .str.extract('([0-9]+\.?[0-9]*)', expand=False)
)
df['TravelledDistance_num'] = pd.to_numeric(df['TravelledDistance_num'], errors='coerce')

# 5.2 Detecta outliers (p. ej. >99º percentil o <1º)
q_low, q_high = df['TravelledDistance_num'].quantile([0.01, 0.99])
mask_td_out = (df['TravelledDistance_num'] < q_low) | (df['TravelledDistance_num'] > q_high)

# 5.3 Imputación con mediana
med_td = df.loc[~mask_td_out, 'TravelledDistance_num'].median()
df.loc[mask_td_out, 'TravelledDistance_clean'] = med_td
df.loc[~mask_td_out, 'TravelledDistance_clean'] = df.loc[~mask_td_out, 'TravelledDistance_num']

print(f"TravelledDistance outliers: {mask_td_out.sum()} filas, imputadas con mediana = {med_td}")


TimeAlive inválido antes de imputar: 78772 filas
Usaron fallback global: 78482 filas
Outliers restantes tras imputación: 0

TravelledDistance outliers: 1578 filas, imputadas con mediana = 9.938


In [6]:
# 1) Lista de columnas nuevas
new_cols = [
    'TimeAlive_seconds', 'TimeAlive_clean',
    'TravelledDistance_num', 'TravelledDistance_clean'
]

# 2) Verifica que existan en df
miss = [c for c in new_cols if c not in df.columns]
if miss:
    raise KeyError(f"Faltan columnas: {miss}")
print("Todas las columnas nuevas presentes ✔️")

# 3) Convierte booleanos a binario
bool_cols = ['RoundWinner','MatchWinner','Survived','AbnormalMatch']

# Before converting to int, fill any remaining NaN values in bool_cols
# Fill NaN with 0, treating missing boolean values as 'False' or unknown
df[bool_cols] = df[bool_cols].fillna(0)

# Now, convert the columns to integer type
df[bool_cols] = df[bool_cols].astype(int)

# 4) (Opcional) Reordena columnas para agrupar originales y nuevas
cols = list(df.columns)
# por ejemplo: mover las nuevas al final
cols = [c for c in cols if c not in new_cols] + new_cols
df = df[cols]

# 5) Exporta y descarga (Google Colab)
df.to_csv('cleaned_dataset.csv', index=False)
from google.colab import files
files.download('cleaned_dataset.csv')

Todas las columnas nuevas presentes ✔️


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
# —————————————————————————————
# 1) ELIMINAR columnas originales y temporales ya no necesarias
# —————————————————————————————
drop_cols = [
    'TimeAlive',                # raw original
    'TimeAlive_seconds',        # parse intermedio
    'TravelledDistance',        # raw original
    'TravelledDistance_num'     # parse intermedio
]
df = df.drop(columns=[c for c in drop_cols if c in df.columns])

# —————————————————————————————
# 2) FILTRAR rondas completas (10 jugadores)
# —————————————————————————————
# Cuenta cuántas filas hay por (MatchId,RoundId)
round_counts = df.groupby(['MatchId','RoundId']).size()

# Selecciona solo las rondas con exactamente 10 jugadores
valid_rounds = round_counts[round_counts == 10].index

# Filtra el DataFrame para mantener solo esas rondas
df = (
    df
      .set_index(['MatchId','RoundId'])
      .loc[valid_rounds]
      .reset_index()
)

# —————————————————————————————
# 3) REORDENAR columnas (opcional)
# —————————————————————————————
# Mover las columnas limpias al final, por legibilidad
clean_cols = ['TimeAlive_clean','TravelledDistance_clean']
cols = [c for c in df.columns if c not in clean_cols] + clean_cols
df = df[cols]

# —————————————————————————————
# 4) EXPORTAR dataset limpio y completo
# —————————————————————————————
df.to_csv('cleaned_dataset_final.csv', index=False)

# Si estás en Google Colab:
from google.colab import files
files.download('cleaned_dataset_final.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>